In [13]:
import pdb, sys, os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
from typing import Tuple, Dict, Any
from tqdm import tqdm
from termcolor import colored
from loguru import logger
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
import optuna
from optuna.trial import Trial
from config.config_definition import config as config_orig
from utils.misc import grab_gpu, fix_random_seed, parse_args
from utils.logging import (
    create_log_folders,
    save_config_info,
    ContexualLogger,
    save_sdre_imgs,
    log_training_results,
    get_tb_writer,
)
from utils.hyperparameter_tuning import run_optuna, suggest_parameters, report_to_pruner
from utils.checkpoint import (
    update_and_save_result,
    initialize_objectives,
    finalize_objectives,
)

from datasets.data_processing import lmdb_dataloaders, move_data_to_device
from models.temporal_integrators import import_model, load_pretrained_states
from models.optimizers import initialize_optimizer
from models.losses import compute_loss_and_metrics
from utils.misc import PyTorchPhaseManager, convert_torch_to_numpy, StopWatch
from utils.performance_metrics import training_setup, accumulate_performance


In [37]:
config_orig['NUM_WORKERS'] = 10

torch.set_num_threads(np.max([1, config_orig['NUM_WORKERS']]))

print(f"{config_orig['NUM_WORKERS']=}")


with StopWatch(unit='sec'):
    data_loaders = lmdb_dataloaders(config_orig)
                                                            


2023-05-01 03:30:21.825 | INFO     | datasets.data_processing:lmdb_dataloaders:293 - loading data... 
2023-05-01 03:30:21.826 | INFO     | datasets.data_processing:lmdb_dataloaders:295 - If this process takes long, consider setting is_load_onto_memory=False or use LMDBIterableDataset.


config_orig['NUM_WORKERS']=10


2023-05-01 03:30:24.839 | INFO     | utils.misc:__exit__:207 - elapsed time:  2.766229 seconds. ()
2023-05-01 03:30:24.844 | INFO     | utils.misc:__exit__:208 - Memory usage: 1887.94 megabytes


In [38]:
np.max([1, config_orig['NUM_WORKERS']])

10

In [39]:
config_orig['IS_LOAD_ONTO_MEMORY']

True